In [42]:
import lightgbm as lgb
import sys
from os import listdir
import os.path
from os.path import isdir, isfile, join
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
def readCSVfromDirectory(dir):
    print(dir)
    filepaths = []
    for f in listdir(dir):
        path = join(dir, f)
        if isfile(path) and f.endswith(".csv"):
            filepaths.append(path)

    if len(filepaths) == 0:
        exit("Directory does not contain any csv files: %s" % dir)

    filepaths = np.sort(filepaths)
    print("Found %s files." % len(filepaths))

    return filepaths


In [31]:
#############################################
#MARK: - Read Files

SOURCE_DIR = "./data/"


SOURCE_DIR = os.path.abspath(SOURCE_DIR)
if not isdir(SOURCE_DIR):
    exit("That's not a directory!")
else:
    print(SOURCE_DIR)

elems = listdir(SOURCE_DIR)
print(elems)
directories = {}
for e in elems:
    path = join(SOURCE_DIR, e)
    
    if isdir(path):
        filepaths = readCSVfromDirectory(path)

        directories[e] = []
        for filepath in filepaths:
            # the files contain values in [V] and [A]
            # use a current Factor of 10e-3 if given in [mA]
            directories[e].append(pd.read_csv(filepath, header=1))

/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data
['ohp_voll-single.psdata', 'laptop-single', '.DS_Store', 'ohp_voll-laptop', 'lampe-ohp_voll.psdata', 'ohp_halb-single', 'ohp_halb-single.psdata', 'ohp_voll-laptop-bosch-lampe.psdata', 'lampe-ohp_voll', 'ohp_voll-single', 'ohp_halb-laptop-lampe.psdata', 'ohp_halb-laptop.psdata', 'bosch-single', 'lampe-single', 'ohp_voll-laptop-bosch-lampe', 'ohp_halb-laptop', 'ohp_halb-laptop-lampe']
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/laptop-single
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/ohp_voll-laptop
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/ohp_halb-single
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/lampe-ohp_voll
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/ohp_voll-single
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataScienceProjekt/data/bosch-single
Found 256 files.
/Users/Felix/Documents/FH/NILM/DataSc

In [41]:
for (id, dfs) in directories.items():
    print(id, len(dfs))
    print(dfs[0].head(4))
    print(dfs[0].describe())
    print('\n\n')

laptop-single 256
       (ms)       (V)       (A)
0 -0.039234 -8.224738  0.018067
1 -0.018754 -8.224738  0.036012
2  0.001726 -3.753777  0.018067
3  0.022206  0.732444  0.018067
              (ms)          (V)          (A)
count  2445.000000  2445.000000  2445.000000
mean     24.987326    38.888648     0.021008
std      14.457959   222.962687     0.338291
min      -0.039234  -321.848200    -1.293191
25%      12.474046  -178.472200    -0.017823
50%      24.987326    81.362350     0.000122
75%      37.500606   251.609800     0.018067
max      50.013886   314.340600     1.419172



ohp_voll-laptop 256
       (ms)        (V)       (A)
0 -0.033703 -12.710960 -1.078524
1 -0.013223  -8.224738 -1.078524
2  0.007257  -3.753777 -1.078524
3  0.027737  -8.224738 -1.078524
              (ms)          (V)          (A)
count  2445.000000  2445.000000  2445.000000
mean     24.992857    38.879487     0.443960
std      14.457959   221.681810     2.560982
min      -0.033703  -317.362000    -5.035249
25% 

KeyError: 0

#### TODO

- Zeit diskretisieren 0ms = 0 und 20ms = 2445 (neue spalte -> id des datensatzes)
- mA zu A umrechnen (falls vorhanden)
- geht multilabel mit gradient boosting?
- in X und Y aufteilen
- attacke!


In [75]:
X = []

for label, data in directories.items():
    # pro gerät alle daten in ein dataframe
    res = pd.concat(data, sort=False)
    #zu der tabelle eine spalte mit label hinzufügen
    res = res.assign(labels= lambda x: label)
    
    X.append(res)

# dataframes zu einem kombinieren
X = pd.concat(X, sort=False)


# Y = df['label']

In [77]:
len(X), len(Y)

(6259200, 0)

In [78]:
X

,(ms),(V),(A),labels,(mA)
0,-0.039234,-8.224738,0.018067,laptop-single,NaN
1,-0.018754,-8.224738,0.036012,laptop-single,NaN
2,0.001726,-3.753777,0.018067,laptop-single,NaN
3,0.022206,0.732444,0.018067,laptop-single,NaN
4,0.042686,0.732444,0.036012,laptop-single,NaN
5,0.063166,0.732444,0.018067,laptop-single,NaN
6,0.083646,9.689627,0.018067,laptop-single,NaN
7,0.104126,9.689627,0.036012,laptop-single,NaN
8,0.124606,9.689627,0.018067,laptop-single,NaN
9,0.145086,9.689627,0.018067,laptop-single,NaN


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)